# Library

In [ ]:
# !git clone https://github.com/cvg/LightGlue.git && cd LightGlue
# !python3 -m pip install -e . --break-system-packages

In [ ]:
import os
import io
import sys
import time
import itertools
import shutil
import subprocess
import multiprocessing
import traceback

import pandas as pd
import numpy as np

from datetime import datetime
from IPython.display import clear_output
from scipy.spatial import ConvexHull
from pyntcloud import PyntCloud

# User defined functions
from vggsfm.datasets.imc_helper import (
    read_cameras_binary, write_cameras_text,
    read_images_binary, write_images_text,
    read_points3D_binary, write_points3D_text
    )

from Utilities.utilities import (
    read_points3D, 
    write_ply, 
    calculate_volume, 
    calculate_volume_file
    )

In [ ]:
def delete_multiple_lines(n=1):
    """Delete the last line in the STDOUT."""
    for _ in range(n):
        sys.stdout.write("\x1b[1A")  # cursor up one line
        sys.stdout.write("\x1b[2K")  # delete the last line
        
def clear_screen():
    os.system('cls' if os.name == 'nt' else 'clear')

# Functions

In [ ]:
def run_OpenMVG(path_base, dir_input, dir_output):    
    code_dir = f"{path_base}/openMVG_Build/software/SfM"
    !python3 {code_dir}/SfM_SequentialPipeline.py \
        {dir_input} \
        {dir_output}
    return None


def run_OpenMVS(path_base, dir_input, dir_output):
    code_dir = f"{path_base}/openMVS/scripts/python"
    !python3 {code_dir}/MvgMvsPipeline.py \
        {dir_input} \
        {dir_output} \
        --steps 0 1 2 3 4 5 11 18 19 20 21 # Sequential
        # --steps 0 1 2 3 4 6 11 18 19 20 21 # Global
        
        # --steps 0 1 2 3 4 5 11 18 19
        # --steps 0 1 2 3 4 5 11 18 19 20 21
        # --steps 0 1 2 3 4 5 11 18 19
    return None


def run_OpenMVS_COLMAP(path_base, dir_input, dir_output):
    code_dir = f"{path_base}/openMVS/scripts/python"
    !python3 {code_dir}/MvgMvsPipeline.py \
        {dir_input} \
        {dir_output} \
        --steps 12 13 14 15 16 17 18 19 20 21  
        # --steps 12 13 14 15 16 17 18 19
        
        # --steps 12 13 14 15 16 17 18 19 20 21        
        # --steps 12 13 14 15 16 17 18 19
        # --preset COLMAP \
        # --steps 12 13 14 15 16 17 18 19 20 21
    return None




def run_COLMAP(path_base, dir_input, dir_output):
    DATASET_PATH = dir_input
    path_sparse = f"{DATASET_PATH}/sparse"
    path_dense = f"{DATASET_PATH}/dense"

    if not os.path.exists(path_sparse):
        os.makedirs(path_sparse)

    if not os.path.exists(path_dense):
        os.makedirs(path_dense)
        
    # !mkdir $DATASET_PATH/sparse
    # !mkdir $DATASET_PATH/dense

    ##################################################
    # SFM
    ##################################################
    # Extract features
    !colmap feature_extractor \
        --database_path $DATASET_PATH/database.db \
        --image_path $DATASET_PATH/images \
        --ImageReader.single_camera_per_folder 1 \
        --ImageReader.default_focal_length_factor 0.5 \
        --ImageReader.camera_model OPENCV \
        --SiftExtraction.estimate_affine_shape=true \
        --SiftExtraction.domain_size_pooling=true

    # Matcher (exhaustive/sequential)
    # !colmap exhaustive_matcher \
    #     --database_path $DATASET_PATH/database.db
        
    !colmap sequential_matcher \
        --database_path $DATASET_PATH/database.db \
        --SiftMatching.max_distance 1 \
        --SiftMatching.guided_matching=true

    !colmap point_triangulator \
        --database_path $DATASET_PATH/database.db \
        --image_path $DATASET_PATH/images \
        --input_path $DATASET_PATH/sparse/0 \
        --output_path $DATASET_PATH/sparse/0

    !colmap mapper \
        --database_path $DATASET_PATH/database.db \
        --image_path $DATASET_PATH/images \
        --output_path $DATASET_PATH/sparse

    !colmap image_undistorter \
        --image_path $DATASET_PATH/images \
        --input_path $DATASET_PATH/sparse/0 \
        --output_path $DATASET_PATH/dense \
        --output_type COLMAP \
        --max_image_size 2000

    ##################################################
    # MVS
    ##################################################
    !colmap patch_match_stereo \
        --workspace_path $DATASET_PATH/dense \
        --workspace_format COLMAP \
        --PatchMatchStereo.geom_consistency true

    !colmap stereo_fusion \
        --workspace_path $DATASET_PATH/dense \
        --workspace_format COLMAP \
        --input_type geometric \
        --output_path $DATASET_PATH/dense/fused.ply

    # Very slow
    # !colmap poisson_mesher \
    #     --input_path $DATASET_PATH/dense/fused.ply \
    #     --output_path $DATASET_PATH/dense/meshed-poisson.ply \
    #     --PoissonMeshing.trim 10

    !colmap delaunay_mesher \
        --input_path $DATASET_PATH/dense \
        --output_path $DATASET_PATH/dense/meshed-delaunay.ply
        
    return None
    
    
def run_VGGSFM(path_base, dir_input, dir_output):
    '''
    Note:
    - Limit the amount of images to 16, or else the model will crash
    - Make sure the folder only contains images
    - Make sure LightGlue is installed
    - As LightGlue is installed in default (3.12.7), make sure to use this version when running the code

    Variables
    - fine_tracking=False
    - query_method=sp+sift
    - max_query_pts=4096
    - visual_tracks=True
    - make_reproj_video=True
     extra_pt_pixel_interval=100
    - concat_extra_points=True

    !git clone https://github.com/cvg/LightGlue.git && cd LightGlue
    !python3 -m pip install -e . --break-system-packages

    !python3 --version
    '''

    # Set path
    path = dir_input
    
    !python3 {path_base}/vggsfm/demo.py \
        SCENE_DIR={path}
        
    path = f"{path}/sparse"

    # Read binary files
    # cameras = read_cameras_binary(path + "/cameras.bin")
    # images = read_images_binary(path + "/images.bin")
    points3D = read_points3D_binary(path + "/points3D.bin")

    # Write text files
    # write_cameras_text(cameras, path + "/cameras.txt")
    # write_images_text(images, path + "/images.txt")
    write_points3D_text(points3D, path + "/points3D.txt")

    # Paths to your COLMAP files
    points3D_file = path + "/points3D.txt"
    output_ply_file = path + "/output.ply"

    # Read points3D.txt and write to .ply
    points3D = read_points3D(points3D_file)
    write_ply(points3D, output_ply_file)

    print(f'PLY file saved to {output_ply_file}')
    
    return None
        
        
def run_MVE(path_base, dir_input, dir_output):
    # Set paths
    code_dir = f"{path_base}/mve/apps"
    image_dir = dir_input

    print('Make Scene')
    !{code_dir}/makescene/makescene \
        -i \
        {image_dir}/images \
        {image_dir}/convert
    
    print('SFM')
    !{code_dir}/sfmrecon/sfmrecon \
        {image_dir}/convert
        # --normalize \
        # --always-full-ba \
        # --use-2cam-tracks
    
    print('DM Recon')
    !{code_dir}/dmrecon/dmrecon \
        -s2 \
        {image_dir}/convert
    
    print('Scene 2 Points')
    !{code_dir}/scene2pset/scene2pset \
        -F2 \
        {image_dir}/convert \
        {image_dir}/convert/pset-L2.ply
    
    print('FSS Recon')
    !{code_dir}/fssrecon/fssrecon \
        {image_dir}/convert/pset-L2.ply \
        {image_dir}/convert/surface-L2.ply
    
    print('Mesh Clean')
    !{code_dir}/meshclean/meshclean \
        -t10 \
        {image_dir}/convert/surface-L2.ply \
        {image_dir}/convert/surface-L2-clean.ply

    return None

# Pipeline

In [ ]:
def get_folders_in_directory(directory):
    # Get a list of all files and folders in the specified directory
    items = os.listdir(directory)
    # Filter out the items that are folders
    folders = [item for item in items if os.path.isdir(os.path.join(directory, item))]
    return folders

def list_directory_contents(directory):
    # Get a list of all files and folders in the specified directory
    items = os.listdir(directory)
    return items

def create_path(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
        
def run_with_timeout(target_func, args=(), timeout=600):
    """
    Runs a function with a timeout. If the function takes longer than `timeout` seconds, it's terminated.
    Returns (success: bool, error_message: str or None)
    """
    def wrapper(queue, *args):
        try:
            target_func(*args)
            queue.put((True, None))
        except Exception as e:
            queue.put((False, str(e)))

    queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=wrapper, args=(queue, *args))
    process.start()
    process.join(timeout)

    if process.is_alive():
        process.terminate()
        process.join()
        return False, f"Timeout: {target_func.__name__} exceeded {timeout} seconds"
    
    return queue.get()

# def check_and_run(file_final, run_func, path_base, dir_input, dir_output):
#     """
#     Check if output file already exists. If not, run the processing function.
#     """
#     if os.path.exists(file_final):
#         print('File Exist')
#         return 'File Exist'
#     run_func(path_base, dir_input, dir_output)
#     return ''


def check_and_run(file_final, run_func, path_base, dir_input, dir_output, timeout=600):
    if os.path.exists(file_final):
        print('File Exist')
        return 'File Exist'

    success, message = run_with_timeout(run_func, args=(path_base, dir_input, dir_output), timeout=timeout)
    if not success:
        print('Error:', message)
        return message
    return 'Success'

def dir_generator(path_base, scale_percent, max_frames, path_bg, subject, data, algorithm):
    if algorithm in ['MVG', 'MVG_MVS', 'COLMAP_MVS']:
        dir_input = os.path.join(path_base, 'CADENCE', 'SHAPE_Frames_input', 
                        f'image_input_{scale_percent}_{max_frames}{path_bg}', 
                        subject, data)
        
    else:
        algorithm = algorithm.lower()
        dir_input = os.path.join(path_base, 'CADENCE', 'SHAPE_Frames_input', 
                                f'image_input_{algorithm}{scale_percent}_{max_frames}{path_bg}', 
                                subject, data)
        
    algorithm = algorithm.lower()
    dir_output = os.path.join(path_base, 'CADENCE', 'SHAPE_Frames_output', 
                            f'{algorithm}_{scale_percent}_{max_frames}{path_bg}', 
                            subject, data)
    create_path(dir_output)
    return dir_input, dir_output

In [ ]:
directory_path = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_input/image_input_50_3'
folders = get_folders_in_directory(directory_path)
folders.sort()

print(folders)

In [ ]:
##################################################
# List Algorithm
##################################################
list_algorithm = [
    # 'MVG',
    'MVG_MVS',
    # 'COLMAP_MVS',
    # 'COLMAP',
    # 'VGGSFM', # Use Python 3.12.7
    # 'MVE',
]

##################################################
# Run No Background
##################################################
list_bg = [
    'with_bg', 
    # 'no_bg',
    ]

##################################################
# Scale and Frame
##################################################
list_scale_percent = [100]
# list_max_frames = [3]
list_max_frames = [5]

##################################################
# Run
##################################################
for scale_percent, max_frames in itertools.product(list_scale_percent, list_max_frames):
    print(f'Scale: {scale_percent}, Max Frames: {max_frames}')

    # Get the list of folders in the directory
    directory_path = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_input/image_input_{scale_percent}_{max_frames}'
    folders = get_folders_in_directory(directory_path)
    folders.sort()
    # folders = ['71']
    # folders = ['72']
    # folders = ['73']
    # folders = ['74']
    
    # n = 2
    # folders = list(range(1 + (n-1)*10, 1 + n*10))
    # folders = [str(x) for x in folders]

    for (algorithm, bg) in itertools.product(list_algorithm, list_bg):
        print(folders)
        for folder in folders:
            folder_path = os.path.join(directory_path, folder)
            
            # Get the list of files and folders in the directory
            contents = list_directory_contents(folder_path)
            contents = [x for x in contents if ':Zone.Identifier' not in x]
            subject = folder_path.split('/')[-1].split('.')[0]


            # list_subject_testing = ['8', '9', '17', '19', '22', '67']
            # list_subject_testing = [1, 2, 6, 7, 8, 9, 
            #                         10, 11, 12, 16, 17, 18, 19, 
            #                         21, 22, 23, 24, 27, 29, 
            #                         30, 35, 
            #                         41, 45, 46, 49, 
            #                         51, 55, 56, 59, 
            #                         60, 61, 63, 67, 69]
            # list_subject_testing = [str(x) for x in list_subject_testing]
            
            # if subject in list_subject_testing:
            if True:
                print('')
                print(f'Subject {subject}')
                
                for data in contents:
                                        
                    print(f'Subject {subject} - {data} - {bg} - {algorithm}')

                    # output_buffer = io.StringIO()
                    # sys.stdout = output_buffer

                    # if (subject != '70') and (data != '70D1R'):
                    #     break    
                    ##################################################
                    # If no bg
                    ##################################################
                    if bg == 'with_bg':
                        path_bg = ''
                    else:
                        path_bg = '_no_bg'
                    
                    ##################################################
                    # Time start
                    ##################################################
                    start_time = time.time()
                    
                    ##################################################
                    # Initialization
                    ##################################################
                    error = ''

                    ##################################################
                    # Run SFM + MVS
                    ##################################################
                    try:
                        path_base = f'/home/weiyanpeh/Git/SFM_Related'

                        if algorithm in ['MVG', 'MVG_MVS', 'COLMAP_MVS']:
                            dir_input, dir_output = dir_generator(path_base, scale_percent, max_frames, path_bg, subject, data, algorithm)
                            
                            # If the output already there, dont run again
                            file_final = os.path.join(dir_output, 'mvs', 'scene_dense_mesh_refine.ply') # MVS
                            # file_final = os.path.join(dir_output, 'sfm', 'cloud_and_poses.ply') # MVG

                            if algorithm == 'MVG':
                                error = check_and_run(file_final, run_OpenMVG, path_base, dir_input, dir_output)                 
                            elif algorithm == 'MVG_MVS':
                                error = check_and_run(file_final, run_OpenMVS, path_base, dir_input, dir_output)   
                            elif algorithm == 'COLMAP_MVS':
                                error = check_and_run(file_final, run_OpenMVS_COLMAP, path_base, dir_input, dir_output)
                            else:
                                error = ''
                            

                        elif algorithm == 'COLMAP':
                            dir_input, dir_output = dir_generator(path_base, scale_percent, max_frames, path_bg, subject, data, algorithm)
                            
                            # If the output already there, dont run again
                            file_final = os.path.join(dir_output, 'dense', 'meshed-delaunay.ply')
                            error = check_and_run(file_final, run_COLMAP, path_base, dir_input, dir_output)   
                            
                            
                        elif algorithm == 'VGGSFM':
                            dir_input, dir_output = dir_generator(path_base, scale_percent, max_frames, path_bg, subject, data, algorithm)

                            # If the output already there, dont run again
                            file_final = os.path.join(dir_output, 'sparse', 'output.ply')
                            error = check_and_run(file_final, run_VGGSFM, path_base, dir_input, dir_output)   
                            
                        elif algorithm == 'MVE':
                            dir_input, dir_output = dir_generator(path_base, scale_percent, max_frames, path_bg, subject, data, algorithm)
                            
                            # If the output already there, dont run again
                            file_final = os.path.join(dir_output, 'convert', 'surface-L2-clean.ply')
                            error = check_and_run(file_final, run_MVE, path_base, dir_input, dir_output)   
                            
                        else:
                            error = 'Cannot find algorithm'

                    except Exception as error:
                        print(error)
                        
                
                        
                    ##################################################
                    # Calculate execution time
                    ##################################################
                    end_time = time.time()
                    execution_time = end_time - start_time
                    
                    
                    # sys.stdout = sys.__stdout__
                    # output_string = output_buffer.getvalue()
                    # output_string = output_string.strip()
                    # output_string = output_string.split('\n')
                    # # output_string = output_string[-5:]
                    # # output_string = ' '.join(output_string)
                    # output_string = output_string[-1]
                    output_string = ''
                    
                    ##################################################
                    # Saving results
                    ##################################################
                    now = datetime.now()
                    
                    list_results = []
                    list_results.append([now, data, algorithm, bg, execution_time, error, output_string])

                    # Convert to DataFrame
                    columns = ['Date', 'Image Data', 'Algorithm', 'BG', 'Execution Time', 'Error', 'Output']
                    df_results = pd.DataFrame(list_results, columns=columns)
                    
                    # Check if the file exists
                    path_file = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/results_1_{algorithm}.csv'
                    if os.path.isfile(path_file):
                        df_results.to_csv(path_file, mode='a', header=False, index=False)
                    else:
                        df_results.to_csv(path_file, mode='w', header=True, index=False)
                    
                    # Clear the list_results list for the next iteration
                    list_results = []
                    
                    # clear_output(wait=True)
                    


In [ ]:
output_string